In [60]:
#returns the dataset
def loadDataSet():
    return [['A', 'B'], ['B', 'D'], ['B', 'C'], ['A', 'B', 'D'], ['A', 'C'], ['B', 'C'], ['A', 'C'], ['A', 'B', 'C', 'E'], ['A', 'B', 'C']]
    # return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [61]:
#C1 is the first phase where we find all unique items in the given dataset
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we can use it as a key in a dict 

In [62]:
#scans the dataset and finds the support for each unique item in the given dataset
#D - dataset, Ck -  a list of candidate sets, minSupport-
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    #retList contains all items in dataset that have a support greater than minSupport
    return retList, supportData

In [63]:
#loads the dataset
dataSet = loadDataSet() 
dataSet

[['A', 'B'],
 ['B', 'D'],
 ['B', 'C'],
 ['A', 'B', 'D'],
 ['A', 'C'],
 ['B', 'C'],
 ['A', 'C'],
 ['A', 'B', 'C', 'E'],
 ['A', 'B', 'C']]

In [64]:
#creates unique list of elements from dataset
C1 = createC1(dataSet)
C1

[frozenset({'A'}),
 frozenset({'B'}),
 frozenset({'C'}),
 frozenset({'D'}),
 frozenset({'E'})]

In [65]:
#D is a dataset in the setform.
D = list(map(set,dataSet))
D

[{'A', 'B'},
 {'B', 'D'},
 {'B', 'C'},
 {'A', 'B', 'D'},
 {'A', 'C'},
 {'B', 'C'},
 {'A', 'C'},
 {'A', 'B', 'C', 'E'},
 {'A', 'B', 'C'}]

In [66]:
#L1 - list of items that have a support greater than minSupport
#suppData - contains unique items and their support values
L1,suppData = scanD(D,C1,2)
L1

[frozenset({'C'}), frozenset({'D'}), frozenset({'B'}), frozenset({'A'})]

In [67]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

In [70]:
def apriori(dataSet, minSupport = 2):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [71]:
L,suppData = apriori(dataSet, 2)
L #list of items which has support greater than minSupport

[[frozenset({'C'}), frozenset({'D'}), frozenset({'B'}), frozenset({'A'})],
 [frozenset({'A', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'D'}),
  frozenset({'A', 'B'})],
 [frozenset({'A', 'B', 'C'})],
 []]

In [76]:
def generateRules(L, supportData, minConf=0.5):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(2, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList 

In [77]:
def calcConf(freqSet, H, supportData, brl, minConf=0.5):
    tempH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            tempH.append(conseq)
    return tempH

In [78]:
rules= generateRules(L,suppData, minConf=0.5)
#all associative rules which have confidence greater than 50%

frozenset({'B', 'C'}) --> frozenset({'A'}) conf: 0.5
frozenset({'A', 'C'}) --> frozenset({'B'}) conf: 0.5
frozenset({'A', 'B'}) --> frozenset({'C'}) conf: 0.5
